In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from semantics_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
file_number = 'a10'

### Read Document

In [4]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [5]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [6]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [7]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [8]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

### Get Coref for Word Embedding Similarity by NP

In [9]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [10]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging dictionaries

In [11]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [12]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

In [13]:
coref_final

{'X10': [[A safety stand-down, 25, 'the stand-down'],
  [A safety stand-down, 26, 'the standdown']],
 'X12': [['Carrier', 23, 'carrier'], ['Carrier', 29, 'carrier']],
 'X13': [[these three crashes this year, 30, 'that crash'],
  [these three crashes this year, 35, 'the increases']],
 'X2': [['NAVY', 10, 'Navy'],
  ['NAVY', 12, 'Navy'],
  ['NAVY', 13, 'Navy'],
  ['NAVY', 15, 'Navy'],
  ['NAVY', 19, 'Navy'],
  ['NAVY', 21, 'Navy'],
  ['NAVY', 27, 'Navy'],
  ['NAVY', 30, 'Navy'],
  ['NAVY', 31, 'Navy'],
  ['NAVY', 35, 'Navy'],
  ['NAVY', 36, 'Navy']],
 'X3': [[THIRD RECENT CRASH, 10, "the third ``Tomcat'' crash"],
  [THIRD RECENT CRASH, 19, 'the crash'],
  [THIRD RECENT CRASH, 19, 'the third Tomcat crash'],
  [THIRD RECENT CRASH, 26, 'the latest crash'],
  [THIRD RECENT CRASH, 30, 'that crash'],
  [THIRD RECENT CRASH, 34, 'The crash rate']],
 'X4': [[entire fleet of F-14 airplanes, 26, 'the entire fleet']],
 'X5': [['Thursday', 19, 'Thursday'], ['Thursday', 26, 'Thursday']],
 'X6': [[The 

### Passing the reference dict to Hobbs

In [14]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [15]:
coref_final_with_pro

{'X10': [[A safety stand-down, 25, 'the stand-down'],
  [A safety stand-down, 26, 'the standdown']],
 'X12': [['Carrier', 23, 'carrier'], ['Carrier', 29, 'carrier']],
 'X13': [[these three crashes this year, 30, 'that crash'],
  [these three crashes this year, 35, 'the increases']],
 'X2': [['NAVY', 10, 'Navy'],
  ['NAVY', 12, 'Navy'],
  ['NAVY', 13, 'Navy'],
  ['NAVY', 15, 'Navy'],
  ['NAVY', 19, 'Navy'],
  ['NAVY', 21, 'Navy'],
  ['NAVY', 27, 'Navy'],
  ['NAVY', 30, 'Navy'],
  ['NAVY', 31, 'Navy'],
  ['NAVY', 35, 'Navy'],
  ['NAVY', 36, 'Navy']],
 'X3': [[THIRD RECENT CRASH, 10, "the third ``Tomcat'' crash"],
  [THIRD RECENT CRASH, 19, 'the crash'],
  [THIRD RECENT CRASH, 19, 'the third Tomcat crash'],
  [THIRD RECENT CRASH, 26, 'the latest crash'],
  [THIRD RECENT CRASH, 30, 'that crash'],
  [THIRD RECENT CRASH, 34, 'The crash rate']],
 'X4': [[entire fleet of F-14 airplanes, 26, 'the entire fleet']],
 'X5': [['Thursday', 19, 'Thursday'], ['Thursday', 26, 'Thursday']],
 'X6': [[The 

### Adding those Cluster Heads which were not predicted

In [16]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [[cluster_val[1],int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [17]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

### Making semantic prediction for unpredicted Cluster Heads

In [18]:
# coref_final_with_pro = getCorefDict_meaning_NP(sentence_dict, coref_final_with_pro, 60)
# coref_final_with_pro

### Taking only the head nouns

In [19]:
def takeHeadNouns(coref_final_with_pro):
    
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]
                        
    return coref_final_with_pro

In [20]:
coref_final_with_pro = takeHeadNouns(coref_final_with_pro)

In [21]:
coref_final_with_pro

{'X10': [[A safety stand-down, 25, 'stand-down'],
  [A safety stand-down, 26, 'standdown']],
 'X12': [['Carrier', 23, 'carrier'], ['Carrier', 29, 'carrier']],
 'X13': [[these three crashes this year, 30, 'crash'],
  [these three crashes this year, 35, 'increases']],
 'X2': [['NAVY', 10, 'Navy'],
  ['NAVY', 12, 'Navy'],
  ['NAVY', 13, 'Navy'],
  ['NAVY', 15, 'Navy'],
  ['NAVY', 19, 'Navy'],
  ['NAVY', 21, 'Navy'],
  ['NAVY', 27, 'Navy'],
  ['NAVY', 30, 'Navy'],
  ['NAVY', 31, 'Navy'],
  ['NAVY', 35, 'Navy'],
  ['NAVY', 36, 'Navy']],
 'X3': [[THIRD RECENT CRASH, 10, 'crash'],
  [THIRD RECENT CRASH, 19, 'crash'],
  [THIRD RECENT CRASH, 19, 'crash'],
  [THIRD RECENT CRASH, 26, 'crash'],
  [THIRD RECENT CRASH, 30, 'crash'],
  [THIRD RECENT CRASH, 34, 'rate']],
 'X4': [[entire fleet of F-14 airplanes, 26, 'fleet']],
 'X5': [['Thursday', 19, 'Thursday'], ['Thursday', 26, 'Thursday']],
 'X6': [[The 72-hour ``safety stand-down'', 25, 'stand-down'],
  [The 72-hour ``safety stand-down'', 26, 'sta

### Print Output

In [22]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [23]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">GROUNDED-JETS</COREF>
{2} {}

<COREF ID="X1">02-22</COREF>
{4} {}

<COREF ID="X2">NAVY</COREF>
{10} {Navy}
{12} {Navy}
{13} {Navy}
{15} {Navy}
{19} {Navy}
{21} {Navy}
{27} {Navy}
{30} {Navy}
{31} {Navy}
{35} {Navy}
{36} {Navy}

<COREF ID="X3">THIRD RECENT CRASH</COREF>
{10} {crash}
{19} {crash}
{19} {crash}
{26} {crash}
{30} {crash}
{34} {rate}

<COREF ID="X4">entire fleet of F-14 airplanes</COREF>
{26} {fleet}

<COREF ID="X5">Thursday</COREF>
{19} {Thursday}
{26} {Thursday}

<COREF ID="X6">The 72-hour ``safety stand-down''</COREF>
{25} {stand-down}
{26} {standdown}

<COREF ID="X7">an F-14 fighter</COREF>
{31} {plane}

<COREF ID="X8">the two Naval aviators on board</COREF>
{14} {aviator}

<COREF ID="X9">Defense Department spokesman Kenneth Bacon</COREF>
{24} {Bacon}
{26} {Bacon}
{26} {him}
{26} {he}
{27} {Bacon}
{35} {Bacon}
{36} {Bacon}

<COREF ID="X10">A safety stand-down</COREF>
{25} {stand-down}
{26} {standdown}

<COREF ID="X11">the USS America</COREF>
{22} {}

<CORE

In [24]:
# with open('{}.response'.format(file_number),'w') as f:
#     f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans